In [ ]:
!git clone https://github.com/princeton-vl/RAFT.git

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
%cd RAFT/

In [ ]:
!./download_models.sh

In [ ]:
!python demo.py --model=models/raft-things.pth --path=demo-frames

In [ ]:
%matplotlib notebook

In [ ]:
import sys
sys.path.append('core')

import argparse
import os
import cv2
import glob
import numpy as np
import torch
from PIL import Image

from raft import RAFT
from utils import flow_viz
from utils.utils import InputPadder
import matplotlib.pyplot as plt
from google.colab.patches import  cv2_imshow

DEVICE = 'cuda'

In [ ]:
!unzip /content/video.zip -d /content/

In [ ]:
def viz(img, flo):
    img = img[0].permute(1,2,0).cpu().numpy()
    flo = flo[0].permute(1,2,0).cpu().numpy()
    
    # map flow to rgb image
    flo = flow_viz.flow_to_image(flo)
    img_flo = np.concatenate([img, flo], axis=0)
    #plt.imshow(img_flo / 255.0)
    #plt.show()
    cv2_imshow(img_flo)

In [ ]:
def load_image(imfile):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(DEVICE)



In [ ]:
def demo(args):
    model = torch.nn.DataParallel(RAFT(args))
    model.load_state_dict(torch.load(args.model))

    model = model.module
    model.to(DEVICE)
    model.eval()

    with torch.no_grad():
        images = glob.glob(os.path.join(args.path, '*.png')) + \
                 glob.glob(os.path.join(args.path, '*.jpg'))
        
        images = sorted(images)
        for imfile1, imfile2 in zip(images[:-1], images[1:]):
            image1 = load_image(imfile1)
            print(image1.shape)
            image2 = load_image(imfile2)
            padder = InputPadder(image1.shape)
            image1, image2 = padder.pad(image1, image2)
            print(image1.shape,image2.shape)

            flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)
            viz(image1, flow_up)


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', default="models/raft-things.pth",help="restore checkpoint")
parser.add_argument('--path', default="/content/video", help="dataset for evaluation")
parser.add_argument('--small', action='store_true', help='use small model')
parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
parser.add_argument('--alternate_corr', action='store_true', help='use efficent correlation implementation')
args = parser.parse_args("")

In [ ]:
demo(args)

In [ ]:
model = torch.nn.DataParallel(RAFT(args))
model.load_state_dict(torch.load(args.model))

model = model.module
model.to(DEVICE)
model.eval()

In [ ]:
VIDEO_PATH = '/content/video.mp4'

In [ ]:
from torch.utils.data import Dataset, DataLoader, IterableDataset
from torchvision import transforms, utils
class VideoFrameDataset(IterableDataset):
    """Video Frame dataset."""

    def __init__(self, video_file, root_dir=None, transform=None):
        """
        Args:
            video_file (string): Path to the video file.
            root_dir (string): Directory with all the videos.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.video_file = video_file
        self.root_dir = root_dir
        self.transform = transform
        self.cap = cv2.VideoCapture(self.video_file)
        

    def __iter__(self):
        
        ret, old_frame = self.cap.read()
        num_frames = (int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT)))
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2RGB)
        for num in range(num_frames - 1):
            ret,frame = self.cap.read()
            #frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            #old_gray = frame_gray.copy()
            

            if self.transform:
                x = self.transform(old_frame)
                y = self.transform(frame)
            else:
                x = old_frame
                y = frame
            old_frame = frame.copy()

            yield x, y
    def __exit__(self, exc_type, exc_value, traceback):
        cv2.destroyAllWindows()
        cap.release()

In [ ]:
video_dataset = VideoFrameDataset(VIDEO_PATH)

In [ ]:
loader = DataLoader(video_dataset,batch_size=1)

In [ ]:
FRAMES = 1000
counter = 0
with torch.no_grad():
    for image1,image2 in loader:
        image1 = image1.permute(0,3,1,2).float()
        image2 = image2.permute(0,3,1,2).float()
        
        #image1[None].to(DEVICE)
        #image2[None].to(DEVICE)
        image1 = image1.cuda()
        image2 = image2.cuda()
        padder = InputPadder(image1.shape)
        image1, image2 = padder.pad(image1, image2)
        flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)
        viz(image1, flow_up)
        if counter == FRAMES:
            break
        counter += 1

